# Evaluate Logistic Regression



In [1]:
import numpy as np
import keras
import tensorflow as tf

# import local files
import sys
sys.path.append('pyDevProj/src')
import utils.load as utils
import utils.preprocess as pre

# fix random seed for reproducibility
np.random.seed(42)

DEBUG = False

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [11]:
import pandas as pd
from tabulate import tabulate

file_name = "modelResults.xlsx"
resultsDF = pd.read_excel(file_name, sheet_name="Sheet1") 

print("Test results")
resultsDF[['data set', 'n', 'm', 'epochs', 'error rate',  'Run', 'file', 'date'] ]
#resultsDF.loc[1:2, ['date', 'file', 'data set', 'n', 'm', 'epochs', 'error rate']]

Test results


,data set,n,m,epochs,error rate,Run,file,date
0,debug train,2,7,5000,0.285714,xxx,simpleLogisticRegression.tfModel,2018-01-30
1,small train,60498100,1000,100,0.028000,xxx,simpleLogisticRegression.tfModel,2018-01-30
2,large train,60498100,15300,100,0.040719,fullDataSet_100epochs,simpleLogisticRegression.tfModel,2018-02-03
3,large test,60498100,3826,0,0.039467,fullDataSet_100epochs,simpleLogisticRegression.tfModel,2018-02-16


In [3]:
model_dir = "models/logisticRegression/simpleLogisticRegression/" 
model_name = "AEDWIP"
model_run = "AEDWIP"

if DEBUG:
    # trained on test data. Ran 500 epochs
    model_run = "2018_02_08_02:19:11/"
    model_name = "model.tfModel"    
    X_test, y_test = utils.test_train_data_set()
else:
    model_run = "fullDataSet_100epochs/"
    model_name = "simpleLogisticRegression.tfModel"
    input_file = "data/tumor_normal.h5"
    X_test, y_test = utils.full_test_data_set(input_file)
    X_test = pre.normalizeData( X_test )
    m,n = X_test.shape
    y_test = np.reshape(y_test, (1,m))
      
X_test = np.transpose(X_test)
print("X_test.shape:", X_test.shape)
print("y_test.shape:", y_test.shape)

Testing on full dataset
X_test.shape: (60498, 3826)
y_test.shape: (1, 3826)


In [4]:
model_path = "{}{}".format(model_dir, model_run)
# simpleLogisticRegression.tfModel
print(model_path)

models/logisticRegression/simpleLogisticRegression/fullDataSet_100epochs/


[http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/
](http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/
)

In [5]:
%%time

# clear out any nodes from previous runs
tf.reset_default_graph()
init = tf.global_variables_initializer()

#saver = tf.train.Saver()
with tf.Session() as sess:
    
    # import pre trained model
    model = model_path + model_name + ".meta"
    print("model:", model)
    saver = tf.train.import_meta_graph(model)
    saver.restore(sess,tf.train.latest_checkpoint(model_path))
        
    # access pre trained model's placeholder variables
    graph = tf.get_default_graph()
    
#     print("AEDWIP\n", [n.name for n in graph.as_graph_def().node])

    X = graph.get_tensor_by_name("input_data/X:0")
    y = graph.get_tensor_by_name("input_data/y:0")
    
    if DEBUG:
        w = graph.get_tensor_by_name("model_parameters/w:0")
        b = graph.get_tensor_by_name("model_parameters/b:0")
    

    # access the op we want to run.
    yhat = graph.get_tensor_by_name("Predict/yhat:0")
    error_rate = graph.get_tensor_by_name("Predict/error_rate:0")
    
    if DEBUG:
        z = graph.get_tensor_by_name("Predict/z:0")
        a = graph.get_tensor_by_name("Predict/a:0") # activation
    
    data = { X:X_test, y:y_test}
    fetchResults ={
                "yhat":yhat,
                "error_rate":error_rate
                }
    
    if DEBUG:
        fetchResults.update({"w":w,"b":b,"z":z,"a":a})

    results = sess.run(fetchResults, feed_dict=data)

    print("error_rate:", results['error_rate'])
    
    if DEBUG:
        print("yhat:", results['yhat']) 
        print("w:", results['w'])
        print("b:", results['b'])
        print("z:", results['z'])
        print("a:", results['a'])
    

model: models/logisticRegression/simpleLogisticRegression/fullDataSet_100epochs/simpleLogisticRegression.tfModel.meta
INFO:tensorflow:Restoring parameters from models/logisticRegression/simpleLogisticRegression/fullDataSet_100epochs/simpleLogisticRegression.tfModel
error_rate: [0.03946681]
CPU times: user 5.19 s, sys: 1.34 s, total: 6.53 s
Wall time: 6.54 s


In [6]:
if DEBUG:
    ww = np.array([[0.36739343],[0.15979535]])
    assert np.isclose(results['w'], ww).all()
    assert np.isclose( results['b'], 0.7924019228518157 )

    zz = np.dot(np.transpose(ww), X_test) + results['b']
    assert np.isclose( results['z'],zz ).all()
    
    aa = 1/(1+np.exp(-zz))
    assert np.isclose( results['a'],aa ).all()
    assert np.isclose(0.42857143, results['error_rate'])